# Visualisation of school count in each suburb 

This notebook will generate visualisation of number of school in each suburb

In [1]:
import pandas as pd
import geopandas as gpd
import folium

### Read in the data

In [2]:
# read the listing file that contain school information
listing = pd.read_csv('../data/raw/SCHOOL&PARK_count_with_SA2_with_SCHOOL&PARK_count.csv')

In [3]:
listing.columns

Index(['Location', 'type_property', 'price', 'LT_resident_pcg', 'owner_pcg',
       'family_pcg', 'state', 'rooms', 'bath', 'parking', 'link', 'public_1',
       'public_1_dis', 'public_2', 'public_2_dis', 'private_1',
       'private_1_dis', 'pulic_1_dis', 'pulic_2_dis', 'rent', 'is_furnished',
       'latitude', 'longitude', 'geometry', 'SA2_NAME', 'SA2_CODE',
       'SAL_NAME21', 'SAL_CODE', 'SA2_NAME21', 'SA2_CODE21', 'METRO_TRAIN',
       'REGIONAL_TRAIN', 'Tram_Count', 'METRO_BUS', 'REGIONAL_BUS',
       'REGIONAL_COACH', 'SKYBUS', 'SAL_NAME', 'School_Count', 'Park_Count'],
      dtype='object')

In [4]:
school_count = listing[['SAL_NAME21', 'SAL_CODE', 'School_Count']]

In [5]:
# drop duplicate 
school_count = school_count.drop_duplicates() 

In [6]:
# replace nan column with 0
school_count = school_count.fillna(0)


In [7]:
# rename column
school_count.rename(columns = {"SAL_CODE": "SAL_CODE21"}, inplace = True)

In [8]:
# Path to the SAL files
SAL_file_path = "../data/raw/victoria_region_gdf/SAL_region_gdf.geojson"

# read the SAL file
SAL_gdf = gpd.read_file(SAL_file_path)

In [9]:
# change the type of the file
SAL_gdf['SAL_CODE21'] = SAL_gdf['SAL_CODE21'].astype('int')

In [10]:
# create a JSON object about location and their respective geomtry
geoJSON = SAL_gdf[['SAL_CODE21', 'geometry']].to_json()

In [11]:
# fill in 0 for the suburbs not in the list, if not in the list, 0 school
school_count = pd.merge(school_count, SAL_gdf, on = 'SAL_CODE21', how = 'right')

In [12]:
# feature selection
sal_school_count = school_count[['SAL_NAME21_y', 'SAL_CODE21', 'School_Count']].fillna(0)
sal_school_count

,SAL_NAME21_y,SAL_CODE21,School_Count
0,Abbeyard,20001.0,0.0
1,Abbotsford,20002.0,2.0
2,Aberfeldie,20003.0,4.0
3,Aberfeldy,20004.0,0.0
4,Acheron,20005.0,0.0
...,...,...,...
2939,Yundool,22940.0,0.0
2940,Yuroke,22941.0,0.0
2941,Yuulong,22942.0,0.0
2942,Zeerust,22943.0,0.0


In [13]:
# the top 5 suburbs with the most school
top5 = sal_school_count.sort_values(by = 'School_Count', ascending = False)[0:5]
top5

,SAL_NAME21_y,SAL_CODE21,School_Count
2749,Werribee,22750.0,15.0
946,Frankston,20947.0,15.0
2390,Sunbury,22391.0,14.0
2160,Reservoir,22161.0,14.0
1012,Glen Waverley,21013.0,13.0


In [14]:
# change the type of the code column
SAL_gdf['SAL_CODE21'] = SAL_gdf['SAL_CODE21'].astype('int')

In [15]:
# inner join to get the shape of all the regions in top 5 
top5 = pd.merge(top5, SAL_gdf, on = 'SAL_CODE21', how = 'inner')
top5

,SAL_NAME21_y,SAL_CODE21,School_Count,SAL_NAME21,SHAPE_Area,geometry
0,Werribee,22750.0,15.0,Werribee,0.004393,"POLYGON ((144.67539 -37.91386, 144.67262 -37.9..."
1,Frankston,20947.0,15.0,Frankston,0.002033,"POLYGON ((145.16317 -38.13236, 145.16351 -38.1..."
2,Sunbury,22391.0,14.0,Sunbury,0.013481,"POLYGON ((144.71079 -37.48236, 144.71112 -37.4..."
3,Reservoir,22161.0,14.0,Reservoir,0.001944,"POLYGON ((145.02209 -37.71516, 145.02201 -37.7..."
4,Glen Waverley,21013.0,13.0,Glen Waverley,0.001725,"POLYGON ((145.16448 -37.86142, 145.16514 -37.8..."


In [16]:
# get the centroid for labelling
top5['centroid'] = top5['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))

In [17]:
# plot the price map for number of school
m1 = folium.Map(location=[-37.8136, 144.9631], tiles="Stamen Terrain", zoom_start=10)

folium.Choropleth(
    geo_data=geoJSON,
    data=sal_school_count,
    columns = ['SAL_CODE21','School_Count'],
    key_on='properties.SAL_CODE21',  
    fill_color='YlOrRd', 
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='number of school',
).add_to(m1)


In [18]:
for zone_name, coord in top5[['SAL_NAME21_y', 'centroid']].values:
    folium.Marker(
        location=coord, 
        tooltip=folium.Tooltip(zone_name, permanent=True, direction="below")
    ).add_to(m1)

# File size will be too large if display the map
#m1

In [19]:
m1.save('../plots/School_per_SAL_region.html')
